In [1]:
%attachments

Training Cluster    ML Engine
------------------  -----------
tcspark             spark


In [3]:
%%tcspark --pyspark
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler 
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator  

fields = StructType([
    StructField('SepalLength', DoubleType(), False),
    StructField('SepalWidth', DoubleType(), False),
    StructField('PetalLength', DoubleType(), False),
    StructField('PetalWidth', DoubleType(), False),
    StructField('Name', StringType(), False)
])

loc='nfs' # nfs or dtap

# if there is no protocol or file identifier, it is a hdfs location
if (loc == 'nfs'):
    nfs_repo='file:///bd-fs-mnt/repo/'
    data=nfs_repo+'data/iris.csv'
    model_save_loc=nfs_repo+'models/iris.spark.model'
else:
    data='dtap://TenantStorage/tmp/iris.csv'
    model_save_loc='dtap://TenantStorage/tmp/iris.spark.model'


df = spark.read.csv(data,schema=fields,header="True",sep=",")
#df.show(5)
print("a")
label_encode = StringIndexer(inputCol = 'Name', outputCol = 'label')
assembler = VectorAssembler(inputCols=df.columns[0:4],outputCol = 'features')
scaler = StandardScaler(inputCol='features', outputCol= 'scaledFeatures')
lr = RandomForestClassifier(featuresCol = 'scaledFeatures', labelCol = 'label')
##pipeline = pyspark.ml.Pipeline(stages=[label_encode,assembler,scaler,lr])
pipeline = Pipeline(stages=[label_encode,assembler,scaler,lr])
(train, test) = df.randomSplit([0.8, 0.2]) 
model = pipeline.fit(train)
predict = model.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predict)    
print("b")
print("Accuracy = %g" % (accuracy))
model.save(model_save_loc)

a
b
Accuracy = 0.84
